In [1]:
import albumentations as A
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os
import numpy as np

In [2]:
def clip_boxes(boxes):
    """Ensure all bounding box coordinates are within [0.0, 1.0]."""
    clipped_boxes = []
    for box in boxes:
        x_min, y_min, x_max, y_max, class_id = box
        x_min = np.clip(x_min, 0.0, 1.0)
        y_min = np.clip(y_min, 0.0, 1.0)
        x_max = np.clip(x_max, 0.0, 1.0)
        y_max = np.clip(y_max, 0.0, 1.0)
        clipped_boxes.append((x_min, y_min, x_max, y_max, class_id))
    return clipped_boxes

In [3]:
def save_augmented_image_with_boxes(image, boxes, output_image_folder, output_annotation_folder, 
                                    transform, image_name, aug_type, brightness_range):
    try:
        """Applies augmentation and saves images with corresponding YOLO annotations."""
        os.makedirs(output_image_folder, exist_ok=True)
        os.makedirs(output_annotation_folder, exist_ok=True)

        # Apply transformation
        transformed = transform(image=image, bboxes=boxes, class_labels=[box[0] for box in boxes])
        transformed_image, transformed_boxes = transformed['image'], transformed['bboxes']
        
        # clipped_boxes = clip_boxes(transformed_boxes)

        # Save the transformed image
        output_image_path = os.path.join(
            output_image_folder, f"{image_name}_{aug_type}_brightness_{brightness_range[0]}-{brightness_range[1]}.jpg"
        )
        cv2.imwrite(output_image_path, transformed_image)

        # Save the corresponding YOLO annotations
        output_annotation_path = os.path.join(
            output_annotation_folder, f"{image_name}_{aug_type}_brightness_{brightness_range[0]}-{brightness_range[1]}.txt"
        )
        with open(output_annotation_path, 'w') as f:
            for box in transformed_boxes:
                x_center, y_center, width, height = box
                class_id = 0  # Assuming a single class; adjust if needed.
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

        print(f"Saved image: {output_image_path}")
        print(f"Saved annotation: {output_annotation_path}")
    
    
    except ValueError as e:
        print(f"Error processing image {image_name} with augmentation {aug_type}: {e}")


In [4]:
def apply_augmentation(image, boxes, image_name, brightness_ranges, aug_type, 
                       transform_fn, output_image_folder, output_annotation_folder):
    """Applies augmentation and saves results."""
    for brightness_range in brightness_ranges:
        transform = A.Compose([
            transform_fn(p=1.0),
            A.ColorJitter(brightness=brightness_range, p=1.0)
        ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

        save_augmented_image_with_boxes(
            image, boxes, output_image_folder, output_annotation_folder, 
            transform, image_name, aug_type, brightness_range
        )

In [5]:
def horizontal_flip_color_jitter(image, boxes, image_name, brightness_ranges, 
                                 output_image_folder, output_annotation_folder):
    apply_augmentation(image, boxes, image_name, brightness_ranges, 'hflip', 
                       A.HorizontalFlip, output_image_folder, output_annotation_folder)

In [6]:
def vertical_flip_color_jitter(image, boxes, image_name, brightness_ranges, 
                               output_image_folder, output_annotation_folder):
    apply_augmentation(image, boxes, image_name, brightness_ranges, 'vflip', 
                       A.VerticalFlip, output_image_folder, output_annotation_folder)

In [7]:
def color_jitter(image, boxes, image_name, brightness_ranges, 
                 output_image_folder, output_annotation_folder):
    """Applies only color jitter without flipping."""
    apply_augmentation(image, boxes, image_name, brightness_ranges, 'color_jitter', 
                       A.NoOp, output_image_folder, output_annotation_folder)

In [8]:
def process_folder(image_folder, annotation_folder, brightness_ranges, 
                   output_image_folder, output_annotation_folder):
    """Applies augmentations to all images and their corresponding annotations."""
    for img_file in os.listdir(image_folder):
        if img_file.endswith(('.jpg', '.png', '.jpeg')):
            image_name = os.path.splitext(img_file)[0]
            img_path = os.path.join(image_folder, img_file)

            # Corresponding annotation file
            label_path = os.path.join(annotation_folder, f"{image_name}.txt")
            if not os.path.exists(label_path):
                print(f"Annotation not found for {img_file}, skipping...")
                continue

            # Read image and annotations
            image = cv2.imread(img_path)
            with open(label_path, 'r') as f:
                bboxes = [list(map(float, line.strip().split())) for line in f]

            # Separate class labels and bbox coordinates
            class_labels = [int(b[0]) for b in bboxes]
            boxes = [b[1:] for b in bboxes]

            # Apply augmentations
            horizontal_flip_color_jitter(image, boxes, image_name, brightness_ranges, 
                                         output_image_folder, output_annotation_folder)
            vertical_flip_color_jitter(image, boxes, image_name, brightness_ranges, 
                                       output_image_folder, output_annotation_folder)
            color_jitter(image, boxes, image_name, brightness_ranges, 
                         output_image_folder, output_annotation_folder)

In [9]:
image_folder = ''
annotation_folder = ''
output_image_folder = ''
output_annotation_folder = ''
brightness_ranges = [(0.0, 0.5), (0.8, 1.0), (1.8, 2.0)]

In [10]:
process_folder(image_folder, annotation_folder, brightness_ranges, 
               output_image_folder, output_annotation_folder)

Saved image: /Users/satyapraneeth/Desktop/cv-project/dataset/agumented_images_latest/images/WhatsApp-Image-2024-10-18-at-13-08-54_jpeg.rf.c864ca8b03591b5defb2a483c4f15973_hflip_brightness_0.0-0.5.jpg
Saved annotation: /Users/satyapraneeth/Desktop/cv-project/dataset/agumented_images_latest/labels/WhatsApp-Image-2024-10-18-at-13-08-54_jpeg.rf.c864ca8b03591b5defb2a483c4f15973_hflip_brightness_0.0-0.5.txt
Saved image: /Users/satyapraneeth/Desktop/cv-project/dataset/agumented_images_latest/images/WhatsApp-Image-2024-10-18-at-13-08-54_jpeg.rf.c864ca8b03591b5defb2a483c4f15973_hflip_brightness_0.8-1.0.jpg
Saved annotation: /Users/satyapraneeth/Desktop/cv-project/dataset/agumented_images_latest/labels/WhatsApp-Image-2024-10-18-at-13-08-54_jpeg.rf.c864ca8b03591b5defb2a483c4f15973_hflip_brightness_0.8-1.0.txt
Saved image: /Users/satyapraneeth/Desktop/cv-project/dataset/agumented_images_latest/images/WhatsApp-Image-2024-10-18-at-13-08-54_jpeg.rf.c864ca8b03591b5defb2a483c4f15973_hflip_brightness_1.